In [ ]:
import numpy as np
import pandas as pd
from Gaussian_Naive_Bayes import GaussianNaiveBayes
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.impute import SimpleImputer

In [ ]:
training_data = pd.read_csv('aps_failure_training_set.csv', skiprows=20)
testing_data = pd.read_csv('aps_failure_test_set.csv', skiprows=20)

In [ ]:
class Scania_APS_Failure(GaussianNaiveBayes):
    def __init__(self,training, testing, n_comp, non_na_thresh, discriminant_analysis='qda', rda_p=np.nan):
        
        self.n_comp = n_comp
        self.training = training.copy()
        self.testing = testing.copy()
        
        self.training.replace(to_replace='na', value=np.nan, inplace=True)
        self.testing.replace(to_replace='na', value=np.nan, inplace=True)
        
        self.training.dropna(axis=1, thresh=int(non_na_thresh*self.training.shape[0]), inplace=True)
        self.testing = self.testing[self.training.columns]
        
        training_label = self.training['class']
        testing_label = self.testing['class']
        
        imputer = SimpleImputer()
        imputed_training = imputer.fit_transform(self.training.iloc[:,1:])
        imputed_testing = imputer.fit_transform(self.testing.iloc[:,1:])
        
        self.training = pd.DataFrame(imputed_training, columns=self.training.columns[1:])
        self.testing = pd.DataFrame(imputed_testing, columns=self.testing.columns[1:])
        
        for column in self.training:
            self.training[column] = pd.qcut(x=self.training[column], q=10, duplicates='drop').cat.codes
            self.testing[column] = pd.qcut(x=self.testing[column], q=10, duplicates='drop').cat.codes
        
        self.training = self.__one_hot_encoding(self.training)
        self.testing = self.__one_hot_encoding(self.testing)
        self.testing['ad_000_8'] = np.array([0 for i in range(self.testing.shape[0])])
        
        self.training = (self.training - self.training.mean(axis=0))/self.training.std(axis=0)
        self.testing = (self.testing - self.testing.mean(axis=0))/self.testing.std(axis=0)
        
        self.training.dropna(axis=1, thresh=int(non_na_thresh*self.training.shape[0]), inplace=True)
        self.testing = self.testing[self.training.columns]
        
        self.__eigen_vector_calculation()
        
        self.training = np.matmul(np.array(self.training),self.Q)
        self.testing = np.matmul(np.array(self.testing),self.Q)
        
        self.training, training_label = SMOTE(sampling_strategy='minority').fit_resample(X=self.training, y=training_label)
        self.training = pd.DataFrame(self.training)
        self.training['class'] = training_label
        
        self.testing = pd.DataFrame(self.testing)
        self.testing['class'] = testing_label
        self.testing['class'].dropna(axis=0, inplace=True)
        
        X_cv, X_test, Y_cv, Y_test = train_test_split(self.testing.iloc[:,:-1], self.testing['class'], test_size=1/3)
        
        self.cv = pd.DataFrame(X_cv)
        self.cv['class'] = Y_cv
        
        self.testing = pd.DataFrame(X_test)
        self.testing['class'] = Y_test
        
        super().__init__(discriminant_analysis,rda_p)
        
        del X_cv, X_test, Y_cv, Y_test, column, discriminant_analysis, imputed_testing, imputed_training, imputer, n_comp, non_na_thresh, rda_p, testing_label, training_label
        
        
    def __eigen_vector_calculation(self):
        data = np.array(self.training)
        sigma_hat = (1/self.training.shape[0]) * np.matmul(data.T,data)
        self.Q = np.linalg.svd(sigma_hat)[0][:,:self.n_comp]
        
    def __one_hot_encoding(self, df):
        data_array = list()
        column_name = list()
        for column in df.columns:
            unique = df[column].unique()
            size = len(df[column].unique())
            df[column].replace(to_replace=unique, value=range(size), inplace=True)
            data_array.append(np.eye(size, size)[df[column]])
            for u in unique:
                column_name.append(column + '_' + str(u))
        return pd.DataFrame(np.concatenate(data_array, axis=1), columns=column_name)

In [ ]:
def evaluate(predicted, actual):
    TP = np.count_nonzero((predicted == 'pos') & (actual == 'pos'))
    TN = np.count_nonzero((predicted == 'neg') & (actual == 'neg'))
    FP = np.count_nonzero((predicted == 'pos') & (actual == 'neg'))
    FN = np.count_nonzero((predicted == 'neg') & (actual == 'pos'))
    
    if (TP + TN + FP + FN) == 0:
        accuracy = 0
    else:
        accuracy = (TP + TN)/(TP + TN + FP + FN)
    
    if (TP + FP) == 0:
        precision = 0
    else:
        precision = TP/(TP + FP)
    
    if (TP + FN) == 0:
        recall = 0
    else:
        recall = TP/(TP + FN)
        
    if (precision + recall) == 0:
        f1_score = 0
    else:
        f1_score = (2 * precision * recall)/(precision + recall)
    
    return (accuracy, precision, recall, f1_score)

In [ ]:
Results = dict()
best = {'Accuracy':0, 'Precision':0, 'Recall':0, 'F1 Score':0, 'Best Obj':'obj', 'Parameter':()}

# We will check for various values of N Components in PCA for assumption of Quadratic Discriminant Analysis

In [ ]:
for non_na_thresh in np.arange(0.50, 0.96, 0.05): 
    for n_comp in range(2,537,19):
        obj = Scania_APS_Failure(training_data, testing_data, n_comp, non_na_thresh)
        obj.fit(obj.training.iloc[:,:-1],obj.training['class'])
        obj.predict(obj.cv.iloc[:,:-1])
        Results[('qda', non_na_thresh, n_comp)] = evaluate(obj.predicted_labels, obj.cv['class'])
        if (Results[('qda',  non_na_thresh, n_comp)][0] + Results[('qda',  non_na_thresh, n_comp)][1] + Results[('qda',  non_na_thresh, n_comp)][2]) > (best['Accuracy'] + best['Precision'] + best['Recall']):
            best['Accuracy'] = Results[('qda',  non_na_thresh, n_comp)][0]
            best['Precision'] = Results[('qda',  non_na_thresh, n_comp)][1]
            best['Recall'] = Results[('qda',  non_na_thresh, n_comp)][2]
            best['F1 Score'] = Results[('qda',  non_na_thresh, n_comp)][3]
            best['Best Obj'] = obj
            best['Parameter'] = ('qda',  non_na_thresh, n_comp)
        else:
            del obj

In [ ]:
best

# We will check for various values of N Components in PCA for assumption of Linear Discriminant Analysis

In [ ]:
for non_na_thresh in np.arange(0.50, 0.96, 0.05): 
    for n_comp in np.arange(2,766,14):
        obj = Scania_APS_Failure(training_data, testing_data, n_comp, non_na_thresh,discriminant_analysis='lda')
        obj.fit(obj.training.iloc[:,:-1],obj.training['class'])
        obj.predict(obj.cv.iloc[:,:-1])
        Results[('lda', non_na_thresh, n_comp)] = evaluate(obj.predicted_labels, obj.cv['class'])
        if (Results[('lda',  non_na_thresh, n_comp)][0] + Results[('lda',  non_na_thresh, n_comp)][1] + Results[('lda',  non_na_thresh, n_comp)][2]) > (best['Accuracy'] + best['Precision'] + best['Recall']):
            best['Accuracy'] = Results[('lda',  non_na_thresh, n_comp)][0]
            best['Precision'] = Results[('lda',  non_na_thresh, n_comp)][1]
            best['Recall'] = Results[('lda',  non_na_thresh, n_comp)][2]
            best['F1 Score'] = Results[('lda',  non_na_thresh, n_comp)][3]
            best['Best Obj'] = obj
            best['Parameter'] = ('lda',  non_na_thresh, n_comp)
        else:
            del obj

In [ ]:
best

# We will check for various values of N Components in PCA for assumption of Regularized Discriminant Analysis

In [ ]:
for non_na_thresh in np.arange(0.50, 0.96, 0.05):
    for alpha in np.arange(0.1,1,0.1):
        for gamma in np.arange(0.1,1,0.1):
            for n_comp in range(2,537,19): 
                obj = Scania_APS_Failure(training_data, testing_data, n_comp, non_na_thresh, 'rda', (alpha, gamma))
                obj.fit(obj.training.iloc[:,:-1],obj.training['class'])
                obj.predict(obj.cv.iloc[:,:-1])
                Results[('rda', n_comp, non_na_thresh, alpha, gamma)] = evaluate(obj.predicted_labels, obj.cv['class'])
                if (Results[('rda', n_comp, non_na_thresh, alpha, gamma)][0] + Results[('rda', n_comp, non_na_thresh, alpha, gamma)][1] + Results[('rda', n_comp, non_na_thresh, alpha, gamma)][2]) > (best['Accuracy'] + best['Precision'] + best['Recall']):
                    best['Accuracy'] = Results[('rda', n_comp, non_na_thresh, alpha, gamma)][0]
                    best['Precision'] = Results[('rda', n_comp, non_na_thresh, alpha, gamma)][1]
                    best['Recall'] = Results[('rda', n_comp, non_na_thresh, alpha, gamma)][2]
                    best['F1 Score'] = Results[('rda', n_comp, non_na_thresh, alpha, gamma)][3]
                    best['Best Obj'] = obj
                    best['Parameter'] = ('rda', n_comp, non_na_thresh, alpha, gamma)
                else:
                    del obj

In [ ]:
best

# From over {(29 * 10) + (55 * 10) + (10 * 9 * 9 *29)} = 24330 Hyperparameter combinaitons we find our best model

In [ ]:
best_obj = best['Best Obj']

In [ ]:
best_obj.predict(best_obj.testing.iloc[:,:-1])

In [ ]:
evaluate(best_obj.predicted_labels, best_obj.testing['diagnosis'])